In [31]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from glob import glob
import plotly.graph_objects as go
import plotly.express as px

import umap

from capo.analysis.utils import get_results, aggregate_results

In [63]:
optims = ["CAPO", "EvoPromptGA", "PromptWizard", "OPRO"]
model = "qwen"
dataset = "sst-5"

dfs = []
for optim in optims:
    df = get_results(dataset, model, optim)
    df = df.sort_values(by=["step", "test_score"])

    df_last_step = df.groupby(["seed"]).last()
    df = df[~df.index.isin(df_last_step.index)]
    df = df.drop_duplicates(subset=["prompt"])

    df = df.assign(optim=optim)

    dfs.append(df)

df = pd.concat(dfs)

In [67]:
embedder = SentenceTransformer("BAAI/bge-m3")
dim_reducer = umap.UMAP(n_neighbors=15, n_components=2, metric="cosine")

c:\Users\tzehl\anaconda3\envs\ds\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning:

`huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tzehl\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development



In [68]:
df["clean_prompt"] = df["prompt"].str.split(r"\[Question\]|Input:").apply(lambda x: x[0]).str.strip()
embs = embedder.encode(df["clean_prompt"].values)
embs_2d = dim_reducer.fit_transform(embs)
df["x1"] = embs_2d[:, 0]
df["x2"] = embs_2d[:, 1]
scores = df["test_score"].values

c:\Users\tzehl\anaconda3\envs\ds\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [ ]:
fig = go.Figure()

# Add contour plot
fig.add_trace(
    go.Contour(
        z=scores,
        x=embs_2d[:, 0],
        y=embs_2d[:, 1],
        contours=dict(
            coloring="heatmap",
            showlabels=False,
        ),
        colorscale="RdBu_r",
        showscale=True,
        # colorbar=dict(
        #     title="Score",
        #     x=1.2,  # Move colorbar much further right
        #     y=0.5,
        #     len=0.75,
        #     titleside="right"
        # )
    )
)

# Define different marker symbols for each category
marker_symbols = ['circle', 'star']
colors = ['blue', 'red', 'green', 'purple', 'orange']

for i, category in enumerate(df["optim"].unique()):
    category_df = df[df["optim"] == category]
    
    fig.add_trace(
        go.Scatter(
            x=category_df["x1"], 
            y=category_df["x2"], 
            mode="markers",
            hoverinfo="text",
            text=df["clean_prompt"],
            marker=dict(
                symbol=[marker_symbols[i % len(marker_symbols)] if idx == category_df["test_score"].idxmax() else "circle" for idx in category_df.index],
                size=10,
                color=colors[i % len(colors)],
                line=dict(width=1, color='black'),
                opacity=0.7
            ),
            name=category,
            showlegend=True
        )
    )

# Update layout
fig.update_layout(
    height=800, 
    width=1000,  # Even wider to accommodate legends
    legend=dict(
        itemsizing="constant",
        xanchor="left",
        yanchor="bottom",
        orientation="v"
    ),
)

# Show the figure
fig.show()